In [1]:
import pandas as pd
                    
sales = pd.read_csv('C:/Users/ANNIE CHRISTINA G/Desktop/data spark/dataset/Sales.csv')
customer = pd.read_csv('C:/Users/ANNIE CHRISTINA G/Desktop/data spark/dataset/Customers.csv', encoding='latin1')
# customer = pd.read_csv('C:/Users/ANNIE CHRISTINA G/Desktop/data spark/dataset/Customers.csv', encoding='iso-8859-1')
# or
# customer = pd.read_csv('C:/Users/ANNIE CHRISTINA G/Desktop/data spark/dataset/Customers.csv', encoding='cp1252')
store = pd.read_csv('C:/Users/ANNIE CHRISTINA G/Desktop/data spark/dataset/Stores.csv')
product = pd.read_csv('C:/Users/ANNIE CHRISTINA G/Desktop/data spark/dataset/Products.csv')
exra = pd.read_csv('C:/Users/ANNIE CHRISTINA G/Desktop/data spark/dataset/Exchange_Rates.csv')


In [2]:
sales

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code
0,366000,1,1/1/2016,NaN,265598,10,1304,1,CAD
1,366001,1,1/1/2016,1/13/2016,1269051,0,1048,2,USD
2,366001,2,1/1/2016,1/13/2016,1269051,0,2007,1,USD
3,366002,1,1/1/2016,1/12/2016,266019,0,1106,7,CAD
4,366002,2,1/1/2016,1/12/2016,266019,0,373,1,CAD
...,...,...,...,...,...,...,...,...,...
62879,2243030,1,2/20/2021,NaN,1216913,43,632,3,USD
62880,2243031,1,2/20/2021,2/24/2021,511229,0,98,4,EUR
62881,2243032,1,2/20/2021,2/23/2021,331277,0,1613,2,CAD
62882,2243032,2,2/20/2021,2/23/2021,331277,0,1717,2,CAD


In [5]:
print(customer.isna().sum())
print(product.isna().sum())
print(sales.isna().sum())
print(store.isna().sum())
print(exra.isna().sum())



CustomerKey     0
Gender          0
Name            0
City            0
State Code     10
State           0
Zip Code        0
Country         0
Continent       0
Birthday        0
dtype: int64
ProductKey        0
Product Name      0
Brand             0
Color             0
Unit Cost USD     0
Unit Price USD    0
SubcategoryKey    0
Subcategory       0
CategoryKey       0
Category          0
dtype: int64
Order Number         0
Line Item            0
Order Date           0
Delivery Date    49719
CustomerKey          0
StoreKey             0
ProductKey           0
Quantity             0
Currency Code        0
dtype: int64
StoreKey         0
Country          0
State            0
Square Meters    1
Open Date        0
dtype: int64
Date        0
Currency    0
Exchange    0
dtype: int64


In [6]:
customer.drop(columns=['State Code'], inplace = True)

In [14]:
import pandas as pd
import re


def clean_text(text):
    
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text)
    cleaned_text = cleaned_text.strip()
    return cleaned_text

customer['Name'] = customer['Name'].apply(clean_text)
customer['City'] = customer['City'].apply(clean_text)
customer['Country'] = customer['Country'].apply(clean_text)
customer['Birthday']= pd.to_datetime(customer['Birthday'])
#output_file_path = 'path/to/your/updated_customers_corrected.csv'
customer.to_csv('customer.csv', index=False)




Names and cities have been cleaned and state codes corrected. Saved to:


In [15]:
##exchange rate
exra['Date']=pd.to_datetime(exra['Date'])
exra.to_csv('exchange_rate.csv', index=False)

In [21]:
##store
store['Square Meters']=store['Square Meters'].fillna(store['Square Meters'].mode()[0])
store['Open Date']=pd.to_datetime(store['Open Date'])
store.to_csv('storeup.csv', index=False)

In [23]:
print(store.isna().sum())

StoreKey         0
Country          0
State            0
Square Meters    0
Open Date        0
dtype: int64


In [27]:
##products
product=product.dropna(subset=['Unit Cost USD'])
product['Unit Cost USD'] = product['Unit Cost USD'].str.replace('$','').str.replace(',','').str.strip().astype(float)
product=product.dropna(subset=['Unit Price USD'])
product['Unit Price USD'] = product['Unit Price USD'].str.replace('$','').str.replace(',','').str.strip().astype(float)
product.to_csv("productup.csv",index = False)


In [30]:
##sales
sales['Delivery Date']=sales['Delivery Date'].fillna(sales['Delivery Date'].mode()[0])
sales[['Order Date', 'Delivery Date']] = sales[['Order Date', 'Delivery Date']].apply(pd.to_datetime)
sales.to_csv('salesuo.csv', index=False)


In [1]:
!pip install mysql-connector-python



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import mysql.connector

db = mysql.connector.connect(
    host='localhost',
    user='root',
    password='7418736408',
    database='redbus_data'
)


cursor = db.cursor()

In [10]:
##customer
cust = """
    CREATE TABLE customer (
    CustomerKey INT PRIMARY KEY,
    Gender VARCHAR(10),
    Name VARCHAR(255),
    City VARCHAR(255),
    State VARCHAR(255),
    Zip_Code VARCHAR(10),
    Country VARCHAR(50),
    Continent VARCHAR(50),
    Birthday DATE
);
"""
cursor.execute(cust)
db.commit()

In [8]:
##exchange

exrat="""
    CREATE TABLE exchange(
    Exchange_Date DATE,
    Currency VARCHAR(10),
    Exchange DECIMAL(10, 4)
);
"""
cursor.execute(exrat)
db.commit()

In [9]:
file_path = 'C:/Users/ANNIE CHRISTINA G/Desktop/data spark/exchange_rate.csv'
exchange_rates_df = pd.read_csv(file_path)


exchange_rates_df.rename(columns={"Date": "Exchange_Date"}, inplace=True)



In [10]:
for i, row in exchange_rates_df.iterrows():
    sql = """
    INSERT INTO exchange (Exchange_Date, Currency, Exchange)
    VALUES (%s, %s, %s)
    """
    cursor.execute(sql, tuple(row))

db.commit()


In [17]:
##product

prod="""
    CREATE TABLE products (
    ProductKey INT PRIMARY KEY,
    Product_Name VARCHAR(255),
    Brand VARCHAR(255),
    Color VARCHAR(50),
    Unit_Cost_USD DECIMAL(10, 2),
    Unit_Price_USD DECIMAL(10, 2),
    SubcategoryKey INT,
    Subcategory VARCHAR(255),
    CategoryKey INT,
    Category VARCHAR(255)
);
"""
cursor.execute(prod)
db.commit()

In [18]:
#sales
sale = """
    CREATE TABLE sales (
    Order_Number INT,
    Line_Item INT,
    Order_Date DATE,
    Delivery_Date DATE,
    CustomerKey INT,
    StoreKey INT,
    ProductKey INT,
    Quantity INT,
    Currency_Code VARCHAR(10)
);
"""
cursor.execute(sale)
db.commit()


In [20]:
##store

stor = """
    CREATE TABLE stores (
    StoreKey INT PRIMARY KEY,
    Country VARCHAR(50),
    State VARCHAR(100),
    Square_Meters DECIMAL(10, 2),
    Open_Date DATE
);
"""
cursor.execute(stor)
db.commit()



In [21]:
##customer
customers_df = pd.read_csv('C:/Users/ANNIE CHRISTINA G/Desktop/data spark/customer.csv')


for i, row in customers_df.iterrows():
    sql = """
    INSERT INTO customers (CustomerKey, Gender, Name, City, State, Zip_Code, Country, Continent, Birthday)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor.execute(sql, tuple(row))

db.commit()



In [23]:
##product
product_df = pd.read_csv('C:/Users/ANNIE CHRISTINA G/Desktop/data spark/productup.csv')

for i, row in product_df.iterrows():
    sql = """
    INSERT INTO products (ProductKey, Product_Name, Brand, Color, Unit_Cost_USD, Unit_Price_USD, SubcategoryKey, Subcategory, CategoryKey, Category)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor.execute(sql, tuple(row))

db.commit()

In [3]:
##exchage

exchange_rates_df = pd.read_csv('C:/Users/ANNIE CHRISTINA G/Desktop/data spark/exchange_rate.csv')

for i, row in exchange_rates_df.iterrows():
    sql = """
    INSERT INTO exchange_rates (Date, Currency, Exchange)
    VALUES (%s, %s, %s)
    """
db.commit()

In [27]:
##sales
sales_df = pd.read_csv('C:/Users/ANNIE CHRISTINA G/Desktop/data spark/salesuo.csv')
 

for i, row in sales_df.iterrows():
    sql = """
    INSERT INTO sales (Order_Number, Line_Item, Order_Date, Delivery_Date, CustomerKey, StoreKey, ProductKey, Quantity, Currency_Code)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor.execute(sql, tuple(row))

db.commit()

In [28]:
##store
stores_df = pd.read_csv('C:/Users/ANNIE CHRISTINA G/Desktop/data spark/storeup.csv')
 
for i, row in stores_df.iterrows():
    sql = """
    INSERT INTO stores (StoreKey, Country, State, Square_Meters, Open_Date)
    VALUES (%s, %s, %s, %s, %s)
    """
    cursor.execute(sql, tuple(row))

db.commit()